In [1]:
import pandas as pd
import numpy as np 

from datetime import timedelta

In [2]:
df = pd.read_excel("./data/ESO 2020Q1.xls", header = 2)

In [3]:
# create dictionary of trimmed incident type numbers

incident_types = {
    100 : "Fire", 
    200 : "Overpressure explosion, overheat - no fire", 
    300 : "Rescue and EMS incidents", 
    400 : "Hazardous Conditions - no fire", 
    500 : "Service Call", 
    600 : "Good intent calls", 
    700 : "False alarms and false calls", 
    800 : "Severe Weather and natural disaster", 
    900 : "Special incident type"
}

In [4]:
df.head()

,Incident Number,Alarm Date,Station,Actions Taken,Aid Given Or Received,Address,Apparatus Name,PSAP Received Date,Dispatched Date,En Route Date,...,Clear Date,Personnel Count,Priority,Type of Response Delay,Is Cancelled Prior To Arrival,Incident Type,Incident Type Code,Shift,Longitude,Latitude
0,20-0324,2020-02-13 15:26:37,1-3,Assist physically disabled,None,"1237 GERDES, Camano Island, Washington",AID103,2020-02-13 15:25:56,2020-02-13 15:26:37,2020-02-13 15:26:58,...,2020-02-13 15:42:29,2.0,Non-Emergent,NaN,NaN,Assist invalid,554.0,Day,-122.471857,48.255921
1,20-0345,2020-02-17 02:15:28,1-2,Provide first aid & check for injuries,None,"1467 RIDGE, Camano Island, Washington",AID103,2020-02-17 02:13:58,2020-02-17 02:15:28,2020-02-17 02:17:06,...,2020-02-17 02:56:01,2.0,Emergent,NaN,NaN,"EMS call, excluding vehicle accident with injury",321.0,B Shift,-122.487304,48.157416
2,20-0345,2020-02-17 02:15:28,1-2,Provide first aid & check for injuries,None,"1467 RIDGE, Camano Island, Washington",MED102,2020-02-17 02:13:58,2020-02-17 02:15:28,2020-02-17 02:16:08,...,2020-02-17 02:56:01,2.0,Emergent,NaN,NaN,"EMS call, excluding vehicle accident with injury",321.0,B Shift,-122.487304,48.157416
3,20-0371,2020-02-21 19:32:35,1-4,Provide basic life support (BLS) Transport per...,None,"731 CHERYL ANN, Camano Island, Washington",AID103,2020-02-21 19:31:50,2020-02-21 19:32:35,2020-02-21 19:33:32,...,2020-02-21 21:03:42,2.0,Emergent,None/No Delay,NaN,"EMS call, excluding vehicle accident with injury",321.0,B Shift,-122.523413,48.186637
4,20-0371,2020-02-21 19:32:35,1-4,Provide basic life support (BLS) Transport per...,None,"731 CHERYL ANN, Camano Island, Washington",E14,2020-02-21 19:31:50,2020-02-21 19:32:35,2020-02-21 19:33:51,...,2020-02-21 19:57:37,3.0,Emergent,None/No Delay,NaN,"EMS call, excluding vehicle accident with injury",321.0,B Shift,-122.523413,48.186637


In [5]:
df.dtypes

Incident Number                          object
Alarm Date                       datetime64[ns]
Station                                  object
Actions Taken                            object
Aid Given Or Received                    object
Address                                  object
Apparatus Name                           object
PSAP Received Date               datetime64[ns]
Dispatched Date                  datetime64[ns]
En Route Date                    datetime64[ns]
Arrival Date                     datetime64[ns]
Clear Date                       datetime64[ns]
Personnel Count                         float64
Priority                                 object
Type of Response Delay                   object
Is Cancelled Prior To Arrival            object
Incident Type                            object
Incident Type Code                      float64
Shift                                    object
Longitude                               float64
Latitude                                

In [6]:
df.isnull().sum()

Incident Number                     2
Alarm Date                          2
Station                             2
Actions Taken                       2
Aid Given Or Received               2
Address                             2
Apparatus Name                      2
PSAP Received Date                  2
Dispatched Date                     2
En Route Date                      40
Arrival Date                      258
Clear Date                          2
Personnel Count                     2
Priority                            2
Type of Response Delay           1202
Is Cancelled Prior To Arrival    1063
Incident Type                       2
Incident Type Code                  2
Shift                               2
Longitude                          14
Latitude                           14
dtype: int64

In [7]:
df = df[df["Incident Number"].isnull() == False].copy()

In [8]:
# calculate time lapse intervals for incidents

df["Turn Out Time Elapsed"] = df["En Route Date"] - df["Dispatched Date"]

df["Travel Time Elapsed"] = df["Arrival Date"] - df["En Route Date"]

df["Response Time Elapsed"] = df["Arrival Date"] - df["Dispatched Date"]

df["Onsite Time Elapsed"] = df["Clear Date"] - df["Arrival Date"]

df["Total Incident Time Elapsed"] = sum([df["Turn Out Time Elapsed"], df["Travel Time Elapsed"], df["Onsite Time Elapsed"]])


In [9]:
# get trimmed incident type code

df["Incident Type Code Trimmed"] = [int(str(df["Incident Type Code"][i])[0] + "00") for i in df.index]

df["Incident Type Trimmed"] = [incident_types[i] for i in df["Incident Type Code Trimmed"]]

In [10]:
# note turn out times that meet goal 
df["Turn Out Goal Met"] = ["YES" if df["Response Time Elapsed"][i] <= timedelta(minutes = 9, seconds = 30) else "NO" for i in df.index]

In [36]:
df.head(20)

,Incident Number,Alarm Date,Station,Actions Taken,Aid Given Or Received,Address,Apparatus Name,PSAP Received Date,Dispatched Date,En Route Date,...,Onsite Time Elapsed,Total Incident Time Elapsed,Incident Type Code Trimmed,Incident Type Trimmed,Turn Out Goal Met,Turn Out Time Elapsed (seconds),Travel Time Elapsed (seconds),Response Time Elapsed (seconds),Onsite Time Elapsed (seconds),Total Incident Time Elapsed (seconds)
0,20-0324,2020-02-13 15:26:37,1-3,Assist physically disabled,None,"1237 GERDES, Camano Island, Washington",AID103,2020-02-13 15:25:56,2020-02-13 15:26:37,2020-02-13 15:26:58,...,00:12:55,00:16:13,500,Service Call,YES,21.0,177.0,198.0,775.0,973.0
1,20-0345,2020-02-17 02:15:28,1-2,Provide first aid & check for injuries,None,"1467 RIDGE, Camano Island, Washington",AID103,2020-02-17 02:13:58,2020-02-17 02:15:28,2020-02-17 02:17:06,...,00:27:33,00:42:11,300,Rescue and EMS incidents,NO,98.0,780.0,878.0,1653.0,2531.0
2,20-0345,2020-02-17 02:15:28,1-2,Provide first aid & check for injuries,None,"1467 RIDGE, Camano Island, Washington",MED102,2020-02-17 02:13:58,2020-02-17 02:15:28,2020-02-17 02:16:08,...,00:35:37,00:41:13,300,Rescue and EMS incidents,YES,40.0,296.0,336.0,2137.0,2473.0
3,20-0371,2020-02-21 19:32:35,1-4,Provide basic life support (BLS) Transport per...,None,"731 CHERYL ANN, Camano Island, Washington",AID103,2020-02-21 19:31:50,2020-02-21 19:32:35,2020-02-21 19:33:32,...,01:23:35,01:32:04,300,Rescue and EMS incidents,YES,57.0,452.0,509.0,5015.0,5524.0
4,20-0371,2020-02-21 19:32:35,1-4,Provide basic life support (BLS) Transport per...,None,"731 CHERYL ANN, Camano Island, Washington",E14,2020-02-21 19:31:50,2020-02-21 19:32:35,2020-02-21 19:33:51,...,00:15:37,00:26:18,300,Rescue and EMS incidents,NO,76.0,565.0,641.0,937.0,1578.0
5,20-0371,2020-02-21 19:32:35,1-4,Provide basic life support (BLS) Transport per...,None,"731 CHERYL ANN, Camano Island, Washington",MED104,2020-02-21 19:31:50,2020-02-21 19:32:35,2020-02-21 19:33:51,...,00:15:08,00:26:18,300,Rescue and EMS incidents,NO,76.0,594.0,670.0,908.0,1578.0
6,20-0160,2020-01-21 02:39:35,1-3,Provide advanced life support (ALS) Transport ...,None,"92 UTSALADY, Camano Island, Washington",AID103,2020-01-21 02:38:35,2020-01-21 02:39:35,2020-01-21 02:41:29,...,00:19:28,00:26:20,300,Rescue and EMS incidents,YES,114.0,298.0,412.0,1168.0,1580.0
7,20-0174,2020-01-22 18:17:29,1-2,Control traffic,None,"MYST, Camano Island, Washington",AID103,2020-01-22 18:16:43,2020-01-22 18:17:29,2020-01-22 18:17:30,...,00:02:35,00:02:38,300,Rescue and EMS incidents,YES,1.0,2.0,3.0,155.0,158.0
8,20-0446,2020-03-09 10:23:36,Stanwood,Cancelled en route,Mutual aid given,"9901 272nd Place NW, Stanwood, Washington",AID103,2020-03-09 10:23:36,2020-03-09 10:23:36,2020-03-09 10:23:51,...,NaT,NaT,600,Good intent calls,NO,15.0,NaN,NaN,NaN,NaN
9,20-0515,2020-03-19 05:41:31,1-2,Provide basic life support (BLS),None,"1578 CRESTVIEW, Camano Island, Washington",AID103,2020-03-19 05:41:20,2020-03-19 05:41:31,2020-03-19 05:44:37,...,01:08:06,01:26:11,300,Rescue and EMS incidents,NO,186.0,899.0,1085.0,4086.0,5171.0


In [34]:
# adding columns for elapsed time that is only in seconds 
# Tableau is dumb, and wigs out with elapses of time longer than 24 hours 
# work arounds need elapsed time to be in seconds in the dataset

df["Turn Out Time Elapsed (seconds)"] = [pd.Timedelta(df["Turn Out Time Elapsed"][n]).seconds for n in df.index]

df["Travel Time Elapsed (seconds)"] = [pd.Timedelta(df["Travel Time Elapsed"][n]).seconds for n in df.index]

df["Response Time Elapsed (seconds)"] = [pd.Timedelta(df["Response Time Elapsed"][n]).seconds for n in df.index]

df["Onsite Time Elapsed (seconds)"] = [pd.Timedelta(df["Onsite Time Elapsed"][n]).seconds for n in df.index]

df["Total Incident Time Elapsed (seconds)"] = [pd.Timedelta(df["Total Incident Time Elapsed"][n]).seconds for n in df.index]

In [35]:
# export

df.to_excel("./data/ESO 2020Q1_cleaned.xlsx", index = False)